In [ ]:
import pandas as pd
import numpy as np
import requests as rq
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

In [ ]:
def get_links():
    links = []

    h2_tags = sp.find_all('h2')
    for h2_tag in h2_tags:
        a_tag = h2_tag.find('a')
        if a_tag and 'href' in a_tag.attrs:
            links.append(a_tag['href'])
    return links

def get_article_title():
    try:
        title = sp.find('h1', class_='article-title').text
        return title if title else 0
    except:
        return 0

def get_article_date():
    try:
        date = sp.find('time').text[:10]
        return date
    except:
        return 0

def get_article_summary():
    try:
        div_tag = sp.find_all('div', class_="articleimg")
        for text in div_tag:
            summary = text.get_text()
        return summary 
    except:
        return 0

def get_article_text():
    try:
        text = ''

        div_tags = sp.find_all('div', class_="post-article text-align-left")
        for div_tag in div_tags:
            p_tags = div_tag.find_all('p')
            for p_tag in p_tags:
                text += p_tag.get_text()    
        return text
    except:
        return 0

In [ ]:
column_name = ['報導標題', '報導日期', '報導大意', '報導內文']
df = pd.DataFrame([[0, 0, 0, 0]], columns=column_name)

In [ ]:
report_type = 'supplement'

for page_num in range(1, 2):
    print(f'Page {page_num} start')
    url = 'https://news.pts.org.tw/' + report_type + '?page=' + str(page_num)
    html = rq.get(url)
    sp = soup(html.text, 'lxml')
    links_in_this_page = get_links()
    for link in links_in_this_page:
        html = rq.get(link)
        sp = soup(html.text, 'lxml')
        title = get_article_title()
        date = get_article_date()
        summary = get_article_summary()
        text = get_article_text()
        df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)
    print(f'Page {page_num} end')
    
df = df.drop(0)
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
df.to_csv(f"data/pts_{report_type}.csv", index=False)